# Recommendation System

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import precision_score, recall_score

In [2]:
df = pd.read_csv('./looker.csv', low_memory=False)

In [3]:
df.info(show_counts=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2466231 entries, 0 to 2466230
Data columns (total 37 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   user_id                 165843 non-null   float64
 1   age                     165843 non-null   float64
 2   gender                  165843 non-null   object 
 3   account_state           165843 non-null   object 
 4   account_city            164274 non-null   object 
 5   account_country         165843 non-null   object 
 6   latitude                165843 non-null   float64
 7   longitude               165843 non-null   float64
 8   account_traffic_source  165843 non-null   object 
 9   account_created_at      165843 non-null   object 
 10  order_id                145887 non-null   float64
 11  order_items_id          145887 non-null   float64
 12  status                  145887 non-null   object 
 13  sale_price              145887 non-null   float64
 14  or

In [4]:
# 장바구니 형태 필요
col_list = [
    'user_id',
    'age',
    'gender',
    'account_country',
    'account_traffic_source',
    'account_created_at',
    'order_id',
    'order_items_id',
    'status',
    'sale_price',
    'order_created_at',
    'sess_traffic_source',
    'product_id',
    'category',
    'brand'
]

In [5]:
df = df[col_list].drop_duplicates()

In [6]:
df.info(show_counts=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 194894 entries, 0 to 2465726
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   user_id                 165843 non-null  float64
 1   age                     165843 non-null  float64
 2   gender                  165843 non-null  object 
 3   account_country         165843 non-null  object 
 4   account_traffic_source  165843 non-null  object 
 5   account_created_at      165843 non-null  object 
 6   order_id                145887 non-null  float64
 7   order_items_id          145887 non-null  float64
 8   status                  145887 non-null  object 
 9   sale_price              145887 non-null  float64
 10  order_created_at        145887 non-null  object 
 11  sess_traffic_source     145892 non-null  object 
 12  product_id              174933 non-null  float64
 13  category                174933 non-null  object 
 14  brand                   

In [7]:
df.describe(include='all')

,user_id,age,gender,account_country,account_traffic_source,account_created_at,order_id,order_items_id,status,sale_price,order_created_at,sess_traffic_source,product_id,category,brand
count,165843.000000,165843.000000,165843,165843,165843,165843,145887.000000,145887.000000,145887,145887.000000,145887,145892,174933.000000,174933,174790
unique,NaN,NaN,2,16,5,1841,NaN,NaN,5,NaN,145726,5,NaN,26,2752
top,NaN,NaN,F,China,Search,2024-01-14,NaN,NaN,Shipped,NaN,2021-04-09 23:41:48+00:00,Email,NaN,Intimates,Allegra K
freq,NaN,NaN,83076,56968,116248,812,NaN,NaN,38042,NaN,2,65585,NaN,13179,6124
mean,49964.691166,41.065701,NaN,NaN,NaN,NaN,62581.780755,90882.658098,NaN,59.536110,NaN,NaN,15140.276569,NaN,NaN
std,28843.590039,17.040903,NaN,NaN,NaN,NaN,36132.860037,52479.390587,NaN,66.437740,NaN,NaN,8410.230687,NaN,NaN
min,1.000000,12.000000,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,0.020000,NaN,NaN,1.000000,NaN,NaN
25%,25005.000000,26.000000,NaN,NaN,NaN,NaN,31268.500000,45443.500000,NaN,24.500000,NaN,NaN,7862.000000,NaN,NaN
50%,49932.000000,41.000000,NaN,NaN,NaN,NaN,62512.000000,90854.000000,NaN,39.990002,NaN,NaN,15728.000000,NaN,NaN
75%,74904.000000,56.000000,NaN,NaN,NaN,NaN,93887.500000,136377.500000,NaN,69.949997,NaN,NaN,22388.000000,NaN,NaN


In [8]:
def convert_type_dict(df):
    ideal_dtypes = dict()
    
    for column in df.columns:
        dtype = df[column].dtype
        
        if ('_id' in column) or (dtype == object):
            n_unique = df[column].nunique()
            
            # 값의 종류가 n개 미만일 경우에만 category 형식으로 최적화
            if n_unique > 10:
                ideal_dtypes[column] = 'object'
            else:
                ideal_dtypes[column] = 'category'
        elif dtype != object:
            c_min = df[column].min()
            c_max = df[column].max()

            # 숫자형 데이터 형식 최적화
            if 'price' in column:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    ideal_dtypes[column] = 'float16'
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    ideal_dtypes[column] = 'float32'
                else:
                    ideal_dtypes[column] = 'float64'
            else:
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    ideal_dtypes[column] = 'int8'
                elif c_min > np.iinfo(np.uint8).min and c_max < np.iinfo(np.uint8).max:
                    ideal_dtypes[column] = 'uint8'
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    ideal_dtypes[column] = 'int16'
                elif c_min > np.iinfo(np.uint16).min and c_max < np.iinfo(np.uint16).max:
                    ideal_dtypes[column] = 'uint16'
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    ideal_dtypes[column] = 'int32'
                elif c_min > np.iinfo(np.uint32).min and c_max < np.iinfo(np.uint32).max:
                    ideal_dtypes[column] = 'uint32'
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    ideal_dtypes[column] = 'int64'
                elif c_min > np.iinfo(np.uint64).min and c_max < np.iinfo(np.uint64).max:
                    ideal_dtypes[column] = 'uint64'
            
    return ideal_dtypes

In [9]:
data_types = convert_type_dict(df)
data_types

{'user_id': 'object',
 'age': 'int8',
 'gender': 'category',
 'account_country': 'object',
 'account_traffic_source': 'category',
 'account_created_at': 'object',
 'order_id': 'object',
 'order_items_id': 'object',
 'status': 'category',
 'sale_price': 'float16',
 'order_created_at': 'object',
 'sess_traffic_source': 'category',
 'product_id': 'object',
 'category': 'object',
 'brand': 'object'}

In [10]:
# 구매한 유저 대상
df = df.dropna(subset='user_id')

In [11]:
df = df.astype(data_types)

In [12]:
df.info(show_counts=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 165843 entries, 7 to 2462749
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   user_id                 165843 non-null  object  
 1   age                     165843 non-null  int8    
 2   gender                  165843 non-null  category
 3   account_country         165843 non-null  object  
 4   account_traffic_source  165843 non-null  category
 5   account_created_at      165843 non-null  object  
 6   order_id                145887 non-null  object  
 7   order_items_id          145887 non-null  object  
 8   status                  145887 non-null  category
 9   sale_price              145887 non-null  float16 
 10  order_created_at        145887 non-null  object  
 11  sess_traffic_source     145887 non-null  category
 12  product_id              145887 non-null  object  
 13  category                145887 non-null  object  
 14  brand   

In [13]:
df['account_created_at'] = pd.to_datetime(df['account_created_at'])
df['order_created_at'] = pd.to_datetime(df['order_created_at'], format='ISO8601')

In [14]:
product_name_table = pd.read_csv('./Looker Ecommerce BigQuery Dataset/products.csv')

In [15]:
product_name_table

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,13842,2.51875,Accessories,Low Profile Dyed Cotton Twill Cap - Navy W39S55D,MG,6.25,Women,EBD58B8A3F1D72F4206201DA62FB1204,1
1,13928,2.33835,Accessories,Low Profile Dyed Cotton Twill Cap - Putty W39S55D,MG,5.95,Women,2EAC42424D12436BDD6A5B8A88480CC3,1
2,14115,4.87956,Accessories,Enzyme Regular Solid Army Caps-Black W35S45D,MG,10.99,Women,EE364229B2791D1EF9355708EFF0BA34,1
3,14157,4.64877,Accessories,Enzyme Regular Solid Army Caps-Olive W35S45D (...,MG,10.99,Women,00BD13095D06C20B11A2993CA419D16B,1
4,14273,6.50793,Accessories,Washed Canvas Ivy Cap - Black W11S64C,MG,15.99,Women,F531DC20FDE20B7ADF3A73F52B71D0AF,1
...,...,...,...,...,...,...,...,...,...
29115,5676,12.42338,Pants & Capris,WHAT GOES AROUND COMES AROUND Women's Colette ...,What Goes Around Comes Around,24.17,Women,3A01FC0853EBEBA94FDE4D1CC6FB842A,10
29116,6538,13.67500,Shorts,WHAT GOES AROUND COMES AROUND Women's Mimi Shorts,What Goes Around Comes Around,25.00,Women,741A0099C9AC04C7BFC822CAF7C7459F,10
29117,6712,12.40000,Shorts,WHAT GOES AROUND COMES AROUND Women's Fifi Short,What Goes Around Comes Around,25.00,Women,5726DAF2C9EE0F955ECA58291C26D2F3,10
29118,6821,13.47500,Shorts,WHAT GOES AROUND COMES AROUND Women's Ferguson...,What Goes Around Comes Around,25.00,Women,BA0A4D6ECEA3E9E126DD3B6D77291C97,10


In [16]:
product_name_table = product_name_table.drop(columns=['sku', 'distribution_center_id', 'cost'])
product_name_table = product_name_table.drop_duplicates()
product_name_table['name'] = product_name_table['name'].fillna('')
product_name_table['brand'] = product_name_table['brand'].fillna('')
product_name_table = product_name_table.set_index('id')

In [17]:
product_name_table

,category,name,brand,retail_price,department
id,,,,,
13842,Accessories,Low Profile Dyed Cotton Twill Cap - Navy W39S55D,MG,6.25,Women
13928,Accessories,Low Profile Dyed Cotton Twill Cap - Putty W39S55D,MG,5.95,Women
14115,Accessories,Enzyme Regular Solid Army Caps-Black W35S45D,MG,10.99,Women
14157,Accessories,Enzyme Regular Solid Army Caps-Olive W35S45D (...,MG,10.99,Women
14273,Accessories,Washed Canvas Ivy Cap - Black W11S64C,MG,15.99,Women
...,...,...,...,...,...
5676,Pants & Capris,WHAT GOES AROUND COMES AROUND Women's Colette ...,What Goes Around Comes Around,24.17,Women
6538,Shorts,WHAT GOES AROUND COMES AROUND Women's Mimi Shorts,What Goes Around Comes Around,25.00,Women
6712,Shorts,WHAT GOES AROUND COMES AROUND Women's Fifi Short,What Goes Around Comes Around,25.00,Women


### 전처리
- 유저 데이터: 유저가 주문한 상품 수, 유저가 구매한 상품의 평균 금액, 성별, 나이, …
- 상품 데이터
    - 상품이 팔린 개수, 이 상품을 구매한 유저 수
    - 가격, 카테고리, 브랜드, 여성복/남성복, 상품명 (빈도수 top 20)            
    - 유저의 선호도 (라벨)
        - 주문 내역 중 Shipped/Complete/Processing → 1
        - 주문 내역 중 Returned/Cancelled → 0

In [18]:
df.brand.nunique()

2744

In [19]:
product_name_table.brand.nunique()

2757

In [20]:
# 원핫인코딩으로 컬럼명이 될 때 기호가 포함되면 안됨 ('_'만 가능)
import re
product_name_table=product_name_table.assign(brand=product_name_table.brand.apply(lambda x:re.sub('[^A-Za-z0-9 _]+', '', x)))

In [21]:
product_name_table.brand.nunique()

2747

- 유저 데이터

In [25]:
# 유저 정보
user_info = df[['user_id', 'age', 'gender', 'account_country', 'account_traffic_source', 'account_created_at']].drop_duplicates().set_index('user_id')

# 유저의 구매 데이터에서 유저 피처 추출 (지불 금액 및 구매 수)
user_avg_price = df.groupby('user_id')['sale_price'].mean().rename('User_AvgPrice')
user_min_price = df.groupby('user_id')['sale_price'].min().rename('User_MinPrice')
user_max_price = df.groupby('user_id')['sale_price'].max().rename('User_MaxPrice')

user_num_orders = df.groupby('user_id')['order_items_id'].nunique().rename('User_NumOrder')

In [26]:
# 추출한 유저 피처 결합
user_features = pd.concat([user_info,
                           user_avg_price, 
                           user_min_price, 
                           user_max_price, 
                           user_num_orders
                           ], axis=1)

In [27]:
user_features

,age,gender,account_country,account_traffic_source,account_created_at,User_AvgPrice,User_MinPrice,User_MaxPrice,User_NumOrder
user_id,,,,,,,,,
76411.0,54,F,China,Search,2023-06-30,9.871094,6.781250,12.960938,2
43659.0,20,F,Brasil,Search,2019-07-22,59.937500,59.937500,59.937500,1
34207.0,55,M,China,Search,2021-04-25,76.000000,16.000000,149.000000,8
58580.0,63,M,China,Email,2023-06-23,46.500000,17.000000,68.000000,3
6809.0,43,F,Brasil,Display,2024-01-15,73.734375,40.000000,125.000000,4
...,...,...,...,...,...,...,...,...,...
78205.0,19,F,China,Facebook,2022-01-06,NaN,NaN,NaN,0
46626.0,63,M,China,Search,2020-01-16,29.984375,29.984375,29.984375,1
23086.0,55,M,Australia,Search,2023-07-07,23.593750,23.593750,23.593750,1


In [28]:
user_features.isna().sum()

age                           0
gender                        0
account_country               0
account_traffic_source        0
account_created_at            0
User_AvgPrice             19956
User_MinPrice             19956
User_MaxPrice             19956
User_NumOrder                 0
dtype: int64

In [30]:
user_features['User_AvgPrice'] = user_features['User_AvgPrice'].fillna(0)
user_features['User_MinPrice'] = user_features['User_MinPrice'].fillna(0)
user_features['User_MaxPrice'] = user_features['User_MaxPrice'].fillna(0)

- 상품 데이터
    - 상품이 팔린 개수, 이 상품을 구매한 유저 수
    - 가격, 카테고리, 브랜드, 여성복/남성복, 상품명 (빈도수 top 20)

In [31]:
product_name_table

,category,name,brand,retail_price,department
id,,,,,
13842,Accessories,Low Profile Dyed Cotton Twill Cap - Navy W39S55D,MG,6.25,Women
13928,Accessories,Low Profile Dyed Cotton Twill Cap - Putty W39S55D,MG,5.95,Women
14115,Accessories,Enzyme Regular Solid Army Caps-Black W35S45D,MG,10.99,Women
14157,Accessories,Enzyme Regular Solid Army Caps-Olive W35S45D (...,MG,10.99,Women
14273,Accessories,Washed Canvas Ivy Cap - Black W11S64C,MG,15.99,Women
...,...,...,...,...,...
5676,Pants & Capris,WHAT GOES AROUND COMES AROUND Women's Colette ...,What Goes Around Comes Around,24.17,Women
6538,Shorts,WHAT GOES AROUND COMES AROUND Women's Mimi Shorts,What Goes Around Comes Around,25.00,Women
6712,Shorts,WHAT GOES AROUND COMES AROUND Women's Fifi Short,What Goes Around Comes Around,25.00,Women


In [32]:
# 상품 정보 외, 상품의 구매 데이터에서 상품 피처 추출 (팔린 횟수 및 구매한 유저 수)
product_num_orders = df.groupby('product_id')['order_items_id'].nunique().rename('product_NumOrder')
product_num_users = df.groupby('product_id')['user_id'].nunique().rename('product_NumOrderUser')

# 추출한 상품 피처 결합
product_features = pd.concat([product_num_orders, product_num_users], axis=1)

product_features = product_features.merge(product_name_table[['department']], how='left', left_index=True, right_index=True)

In [34]:
# 상품명 전처리
import re

def rmv_attr(x):
    if x.isnull().values.any(): # brand가 NaN
        return x
    else:
        result = x['name']

        brand_name = re.compile(re.escape(x['brand']), re.IGNORECASE)
        if '&' in x['category']:
            cat_name_list = x['category'].split('&')
        else:
            cat_name_list = [x['category']]

        cat_compiles = []
        for cat_name in cat_name_list:
            if cat_name.endswith('es'):
                cat_name = cat_name[:-2]
            elif cat_name.endswith('s'):
                cat_name = cat_name[:-1]
            category = re.compile(re.escape(cat_name), re.IGNORECASE)
            cat_compiles.append(category)
        
        department = re.compile(re.escape(x['department']), re.IGNORECASE)

        result = brand_name.sub('', result)  # 브랜드 이름 제거

        for cat_compile in cat_compiles:
            result = cat_compile.sub('', result)
        result = department.sub('', result)
        x['name'] = result
        return x

In [35]:
# 상품명 토큰화
from collections import Counter

documents = product_name_table[product_name_table.index.isin(product_features.index)].drop(columns='retail_price').apply(rmv_attr, axis=1)['name'].values

# Preprocess sentences: lower case and tokenize
documents = [re.sub('[^A-Za-z]', ' ', str(doc)) for doc in documents]    # 알파벳만 남기기
tokenized_documents = [re.findall('[A-Za-z]\w+', doc.lower()) for doc in documents]   # 알파벳 하나만 있는 단어 외 모든 단어를 토큰화

In [37]:
product_features['product_name'] = tokenized_documents

In [41]:
product_features = product_features.merge(product_name_table[['brand', 'category']], left_index=True, right_index=True, how='left')

In [42]:
product_features

,product_NumOrder,product_NumOrderUser,department,product_name,brand,category
product_id,,,,,,
1.0,3,3,Women,"[low, profile, dyed, cotton, twill, cap, navy]",Seven7,Tops & Tees
2.0,4,4,Women,"[low, profile, dyed, cotton, twill, cap, putty]",Calvin Klein,Tops & Tees
3.0,3,3,Women,"[enzyme, regular, solid, army, caps, black]",Calvin Klein Jeans,Tops & Tees
4.0,4,4,Women,"[enzyme, regular, solid, army, caps, olive, on...",Bailey 44,Tops & Tees
5.0,5,5,Women,"[washed, canvas, ivy, cap, black]",Anne Klein,Tops & Tees
...,...,...,...,...,...,...
29116.0,9,9,Men,"[colette, pants]",RayBan,Accessories
29117.0,5,5,Men,[mimi],TUMI,Accessories
29118.0,3,3,Men,[fifi],SPY,Accessories


In [43]:
user_features

,age,gender,account_country,account_traffic_source,account_created_at,User_AvgPrice,User_MinPrice,User_MaxPrice,User_NumOrder
user_id,,,,,,,,,
76411.0,54,F,China,Search,2023-06-30,9.871094,6.781250,12.960938,2
43659.0,20,F,Brasil,Search,2019-07-22,59.937500,59.937500,59.937500,1
34207.0,55,M,China,Search,2021-04-25,76.000000,16.000000,149.000000,8
58580.0,63,M,China,Email,2023-06-23,46.500000,17.000000,68.000000,3
6809.0,43,F,Brasil,Display,2024-01-15,73.734375,40.000000,125.000000,4
...,...,...,...,...,...,...,...,...,...
78205.0,19,F,China,Facebook,2022-01-06,0.000000,0.000000,0.000000,0
46626.0,63,M,China,Search,2020-01-16,29.984375,29.984375,29.984375,1
23086.0,55,M,Australia,Search,2023-07-07,23.593750,23.593750,23.593750,1


- 라벨: 유저의 선호도
    - 주문 내역 중 Shipped/Complete/Processing → 1
    - 주문 내역 중 Returned/Cancelled → 0

In [44]:
df = df[['user_id', 'product_id', 'order_created_at', 'status']].drop_duplicates()
df = df.sort_values(['user_id','order_created_at'])    # 과거/최신 데이터로 train_test_split

# df = df.assign(status=df['status'].map({'Shipped':1, 'Processing':1, 'Complete':1, 'Cancelled':0, 'Returned':0}))

df = df.join(user_features, on='user_id')
df = df.merge(product_features, left_on='product_id', right_index=True, how='inner')    # 구매한 유저만

In [46]:
# df.to_csv('./dataset.csv', index=False)
df

,user_id,product_id,order_created_at,status,age,gender,account_country,account_traffic_source,account_created_at,User_AvgPrice,User_MinPrice,User_MaxPrice,User_NumOrder,product_NumOrder,product_NumOrderUser,department,product_name,brand,category
969940,1.0,7656.0,2022-07-19 11:29:28+00:00,Shipped,62,F,South Korea,Search,2022-03-23,53.328125,15.000000,125.00000,3,6,6,Women,"[buttoned, vest, with, mini, belted, cinched, ...",eVogues Apparel,Blazers & Jackets
553584,1.0,2953.0,2022-07-20 10:32:05+00:00,Shipped,62,F,South Korea,Search,2022-03-23,53.328125,15.000000,125.00000,3,7,7,Women,"[as, medium, smooth, eyeglass, case]",Tommy Hilfiger,Active
1124826,1.0,4731.0,2022-07-20 11:05:38+00:00,Shipped,62,F,South Korea,Search,2022-03-23,53.328125,15.000000,125.00000,3,5,5,Women,"[dandy, slouchy, straight, leg]",Joes Jeans,Jeans
1824631,2.0,25774.0,2022-02-20 10:28:57+00:00,Complete,65,M,Brasil,Organic,2020-10-19,22.000000,22.000000,22.00000,1,5,5,Men,"[ladies, long, sleeve, front, opening, turn, d...",Tommy Bahama,Underwear
1882634,3.0,18177.0,2023-03-10 07:14:45+00:00,Shipped,16,M,United States,Organic,2023-03-09,84.562500,59.500000,110.00000,4,7,7,Men,"[plaid, flannel, sleepwear, pants]",SmartWool,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407767,99998.0,21923.0,2023-07-22 05:16:01+00:00,Processing,51,M,United States,Search,2022-10-31,38.994793,21.984375,55.00000,3,10,10,Men,"[reversible, merino, wool, tartan, vest, rever...",Allegra K,Pants
585624,99999.0,25959.0,2023-12-23 08:59:09+00:00,Shipped,13,M,Brasil,Search,2019-09-12,33.984375,25.000000,42.96875,2,5,5,Men,"[fashion, single, breasted, pockets, front, si...",Hanes,Underwear
630856,99999.0,20828.0,2023-12-23 11:33:17+00:00,Shipped,13,M,Brasil,Search,2019-09-12,33.984375,25.000000,42.96875,2,11,11,Men,"[ladies, gold, color, studded, black, faux, le...",Wrangler,Jeans
1567675,100000.0,10017.0,2020-08-28 23:27:01+00:00,Processing,69,F,China,Display,2020-06-11,76.250000,22.500000,130.00000,2,5,5,Women,"[nocturnal, elegance, sleepshort]",Paddi Murphy,Sleep & Lounge


### train_test_split
- train:과거 / test:최신
    - 과거 구매 1
    - 과거 구매 1 / 최근 구매 1
    - 과거 구매 2 / 최근 구매 1 `⇒ 여기 까지가 전체 유저의 약 90%`
    - 과거 구매 3 ~ 11 / 최근 구매 1

In [41]:
# 재구매 유저만
#df = df[df['User_NumOrder'] > 1]

In [42]:
df = pd.read_csv('./dataset.csv')

In [47]:
df.shape

(145887, 19)

In [48]:
# 재구매 유저만
test = df[df['User_NumOrder'] > 1].sort_values('order_created_at').groupby('user_id').tail(1)
test = test.drop(columns='order_created_at')
test.shape

(36077, 18)

In [49]:
train = df.drop(index=test.index)
train = train.drop(columns='order_created_at')
train.shape

(109810, 18)

In [50]:
36077 / 145887

0.24729413861413285

In [51]:
# random shuffle
train = train.sample(frac=1,random_state=42)
test = test.sample(frac=1,random_state=42)

In [52]:
train.to_csv('./train.csv', index=False)
test.to_csv('./test.csv', index=False)